In [ ]:
theewaterskloof['month'] = theewaterskloof['date'].dt.month
theewaterskloof_winter = theewaterskloof.loc[(theewaterskloof['month'] >= 6) &
                                            (theewaterskloof['month'] < 10) &
                                            (theewaterskloof['diff_storage_ml'] > 0)]
theewaterskloof_winter['diff_storage_ml'].corr(theewaterskloof_winter['rolling2_lead1_precipm'])
plt.figure(figsize = (10, 7))
plt.scatter(theewaterskloof_winter['diff_storage_ml'],theewaterskloof_winter['rolling2_lead1_precipm'])

In [ ]:
theewaterskloof_winter['precipm'].hist(bins = 30)

In [ ]:
num_var = data.select_dtypes(include=[np.number]).columns
num_var = num_var.drop(['diff_storage_ml', 'storage_ml', 'height_m', 'percdiff_storage_ml'])

In [ ]:
Split data into training & test sets. Will do this based on date:
2012, 2014, 2016: train
2013, 2015, 2017: test

In [ ]:
dtrain = data[data['date'].dt.year.isin([2012, 2014, 2016])].dropna(axis = 0, how = 'any')
dtest = data[data['date'].dt.year.isin([2013, 2015, 2017])].dropna(axis = 0, how = 'any')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

dtrain = data[data['date'].dt.year.isin([2012, 2014, 2016])].dropna(axis = 0, how = 'any')
dtest = data[data['date'].dt.year.isin([2013, 2015, 2017])].dropna(axis = 0, how = 'any')

x_train = dtrain[num_var]
y_train = dtrain['percdiff_storage_ml']

x_test = dtest[num_var]
y_test = dtest['percdiff_storage_ml']

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)
print(regr.feature_importances_)
ypred_rf = regr.predict(x_test)


y_actual_vs_pred = pd.DataFrame({'actual': y_test,
                                 'pred': ypred_rf})
fig = plt.figure(figsize = (15,15))
plt.scatter(y_actual_vs_pred['actual'], y_actual_vs_pred['pred'])

In [ ]:
error(y_actual_vs_pred['actual'], y_actual_vs_pred['pred'], log = False)
importances = regr.feature_importances_
std = np.std([tree.feature_importances_ for tree in regr.estimators_],
             axis=0)
zip_importances = list(zip(x_train.columns, importances))
indices = np.argsort(importances)[::-1]
zip_importances = [zip_importances[i] for i in indices]

# Print the feature ranking
print("Feature ranking:")

for f in range(x_train.shape[1]):
    i = indices[f]
    print("%d. %s (%f)" % (f + 1, zip_importances[i][0], zip_importances[i][1]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
features = x_train.columns
features = [features[i] for i in indices]
plt.xticks(range(x_train.shape[1]), features, rotation = 90)
plt.xlim([-1, x_train.shape[1]])
plt.show();

In [ ]:
import sklearn.linear_model as linear_model

ridge = linear_model.RidgeCV(normalize = True)

x_train = dtrain[num_var]
y_train = dtrain['diff_storage_ml']
ridge.fit(x_train, y_train)

ypred_train = ridge.predict(x_train)
print(error(y_train, ypred_train, log = False))

x_test = dtest.drop('diff_storage_ml', axis = 1)[num_var]

ypred_ridge = ridge.predict(x_test)

y_actual_vs_pred = pd.DataFrame({'actual': dtest['diff_storage_ml'],
                                 'pred': ypred_ridge})
# fig = plt.figure(figsize = (30,30))
sns.lmplot('actual', 'pred', y_actual_vs_pred)